In [2]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

import numpy as np

In [4]:
DATASET_PATH = "/home/tlips/Code/robot-imitation-glue/datasets/pour-cup-backup"
img_name ="scene_image"

dataset = LeRobotDataset("",DATASET_PATH,episodes=np.arange(0,22))



Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
def custom_get_item(dataset, idx, ep_idx = None):
    item = dataset.hf_dataset[idx]
    if not ep_idx:
        ep_idx = item["episode_index"].item()




    query_indices = None
    if dataset.delta_indices is not None:
        query_indices, padding = dataset._get_query_indices(idx, ep_idx)
        query_result = dataset._query_hf_dataset(query_indices)
        item = {**item, **padding}
        for key, val in query_result.items():
            item[key] = val

    if len(dataset.meta.video_keys) > 0:
        current_ts = item["timestamp"].item()
        query_timestamps = dataset._get_query_timestamps(current_ts, query_indices)
        video_frames = dataset._query_videos(query_timestamps, ep_idx)
        item = {**video_frames, **item}

    if dataset.image_transforms is not None:
        image_keys = dataset.meta.camera_keys
        for cam in image_keys:
            item[cam] = dataset.image_transforms(item[cam])

    # Add task as a string
    task_idx = item["task_index"].item()
    item["task"] = dataset.meta.tasks[task_idx]

    return item

In [81]:
custom_get_item(dataset,0,0)

{'wrist_image_original': tensor([[[0.4627, 0.4627, 0.4627,  ..., 0.5725, 0.5765, 0.5843],
          [0.4627, 0.4627, 0.4627,  ..., 0.5725, 0.5765, 0.5843],
          [0.4627, 0.4627, 0.4627,  ..., 0.5725, 0.5765, 0.5843],
          ...,
          [0.1765, 0.1765, 0.1765,  ..., 0.2235, 0.2235, 0.2235],
          [0.1765, 0.1765, 0.1765,  ..., 0.2235, 0.2235, 0.2235],
          [0.1765, 0.1765, 0.1765,  ..., 0.2235, 0.2235, 0.2235]],
 
         [[0.4627, 0.4627, 0.4627,  ..., 0.6039, 0.6078, 0.6157],
          [0.4627, 0.4627, 0.4627,  ..., 0.6039, 0.6078, 0.6157],
          [0.4627, 0.4627, 0.4627,  ..., 0.6039, 0.6078, 0.6157],
          ...,
          [0.2431, 0.2431, 0.2431,  ..., 0.3451, 0.3451, 0.3451],
          [0.2431, 0.2431, 0.2431,  ..., 0.3451, 0.3451, 0.3451],
          [0.2431, 0.2431, 0.2431,  ..., 0.3451, 0.3451, 0.3451]],
 
         [[0.4431, 0.4431, 0.4431,  ..., 0.6235, 0.6275, 0.6353],
          [0.4431, 0.4431, 0.4431,  ..., 0.6235, 0.6275, 0.6353],
          [0.443

In [7]:
new_dataset = LeRobotDataset.create(
        repo_id="",
        fps=dataset.fps,
        root=DATASET_PATH + "-recover",
        features=dataset.features,
        use_videos=True,
        image_writer_processes=0,
        image_writer_threads=8,
    )

In [8]:
start_indices, end_indices = dataset.episode_data_index["from"], dataset.episode_data_index["to"]
for episode_index, (start,end) in enumerate(zip(start_indices,end_indices)):
    print(episode_index)
    for idx in range(start.item(), end.item()):
        frame = custom_get_item(dataset,idx,episode_index)
       # these are auto-generated, so remove them from the frame
        features_to_drop_list = ["index", "timestamp", "frame_index", "episode_index", "task_index"]
        
        frame = {k: v for k, v in frame.items() if k not in features_to_drop_list}
        for key, value in frame.items():
            # drop auto-generated meta data
            # convert 0D tensors to 1D tensors
            if hasattr(value, "shape") and len(value.shape) == 0:
                value = value.unsqueeze(0)
                frame[key] = value
            # make images channel-last again
            if hasattr(value, "shape") and len(value.shape) == 3:
                value = value.permute(1, 2, 0)
                frame[key] = value

        new_dataset.add_frame(frame)

    new_dataset.save_episode()


0


Map:   0%|          | 0/443 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1


Map:   0%|          | 0/424 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/386 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

21


Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [73]:
start_indices, end_indices = dataset.episode_data_index["from"], dataset.episode_data_index["to"]

def visualize_images(start,end):
    import matplotlib.pyplot as plt 
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(start)
    ax1.set_title('Start Image')
    ax1.axis('off')
    ax2.imshow(end) 
    ax2.set_title('End Image')
    ax2.axis('off')
    plt.show()

print(start_indices.shape)
print(len(dataset))
for episode_index, (start,end) in enumerate(zip(start_indices,end_indices)):
    print(f"{episode_index=}")
    # for i in range(start.item(), end.item()):
    #     x = dataset[i]
    # print(episode_index)
    print(start)
    print(end)

    start_img = dataset[start.item()][img_name]
    end = end.item() -1
    end_img = custom_get_item(dataset,end,episode_index)[img_name]

    start_img = start_img.permute(1,2,0).numpy()
    end_img = end_img.permute(1,2,0).numpy()
    #visualize_images(start_img,end_img)

torch.Size([48])
19310
episode_index=0
tensor(0)
tensor(443)
episode_index=1
tensor(443)
tensor(867)
episode_index=2
tensor(867)
tensor(1276)
episode_index=3
tensor(1276)
tensor(1673)
episode_index=4
tensor(1673)
tensor(2004)
episode_index=5
tensor(2004)
tensor(2396)
episode_index=6
tensor(2396)
tensor(2696)
episode_index=7
tensor(2696)
tensor(3063)
episode_index=8
tensor(3063)
tensor(3385)
episode_index=9
tensor(3385)
tensor(3725)
episode_index=10
tensor(3725)
tensor(4038)
episode_index=11
tensor(4038)
tensor(4475)
episode_index=12
tensor(4475)
tensor(4927)
episode_index=13
tensor(4927)
tensor(5365)
episode_index=14
tensor(5365)
tensor(5711)
episode_index=15
tensor(5711)
tensor(5969)
episode_index=16
tensor(5969)
tensor(6264)
episode_index=17
tensor(6264)
tensor(6681)
episode_index=18
tensor(6681)
tensor(7110)
episode_index=19
tensor(7110)
tensor(7644)
episode_index=20
tensor(7644)
tensor(8030)
episode_index=21
tensor(8030)
tensor(8450)
episode_index=22
tensor(8450)
tensor(8898)
episo

RuntimeError: Could not open input file: /home/tlips/Code/robot-imitation-glue/datasets/pour-cup-backup/videos/chunk-000/wrist_image_original/episode_000048.mp4 No such file or directory